In [1]:
from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from threading import Thread
import queue

In [2]:
class TestWrapper(EWrapper):
    """
    The wrapper deals with the action coming back from the IB gateway or TWS instance
    We override methods in EWrapper that will get called when this action happens, like currentTime
    """

    ## error handling code
    def init_error(self):
        error_queue=queue.Queue()
        self._my_errors = error_queue

    def get_error(self, timeout=5):
        if self.is_error():
            try:
                return self._my_errors.get(timeout=timeout)
            except queue.Empty:
                return None

        return None


    def is_error(self):
        an_error_if=not self._my_errors.empty()
        return an_error_if

    def error(self, id, errorCode, errorString):
        ## Overriden method
        errormsg = "IB error id %d errorcode %d string %s" % (id, errorCode, errorString)
        self._my_errors.put(errormsg)

    ## Time telling code
    def init_time(self):
        time_queue=queue.Queue()
        self._time_queue = time_queue

        return time_queue

    def currentTime(self, time_from_server):
        ## Overriden method
        self._time_queue.put(time_from_server)

In [3]:
class TestClient(EClient):
    """
    The client method
    We don't override native methods, but instead call them from our own wrappers
    """
    def __init__(self, wrapper):
        ## Set up with a wrapper inside
        EClient.__init__(self, wrapper)

    def speaking_clock(self):
        """
        Basic example to tell the time
        :return: unix time, as an int
        """

        print("Getting the time from the server... ")

        ## Make a place to store the time we're going to return
        ## This is a queue
        time_storage=self.wrapper.init_time()

        ## This is the native method in EClient, asks the server to send us the time please
        self.reqCurrentTime()

        ## Try and get a valid time
        MAX_WAIT_SECONDS = 10

        try:
            current_time = time_storage.get(timeout=MAX_WAIT_SECONDS)
        except queue.Empty:
            print("Exceeded maximum wait for wrapper to respond")
            current_time = None

        while self.wrapper.is_error():
            print(self.get_error())

        return current_time

In [4]:
class TestApp(TestWrapper, TestClient):
    def __init__(self, ipaddress, portid, clientid):
        TestWrapper.__init__(self)
        TestClient.__init__(self, wrapper=self)

        self.init_error()
        self.connect(ipaddress, portid, clientid)

        thread = Thread(target = self.run)
        thread.start()

        setattr(self, "_thread", thread)


In [31]:
app = TestApp("127.0.0.1", 7497, 0)

In [32]:
current_time = app.speaking_clock()
print(current_time)

Getting the time from the server... 
IB error id -1 errorcode 2104 string Market data farm connection is OK:usfarm.nj
IB error id -1 errorcode 2104 string Market data farm connection is OK:eufarm
IB error id -1 errorcode 2104 string Market data farm connection is OK:cashfarm
IB error id -1 errorcode 2104 string Market data farm connection is OK:usfarm
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:euhmds
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ushmds
1562692731


In [35]:
app.isConnected()

False

In [34]:
app.disconnect()

unhandled exception in EReader thread
Traceback (most recent call last):
  File "c:\users\martin\dev\_pyvenvs\latest36\lib\site-packages\ibapi\reader.py", line 34, in run
    data = self.conn.recvMsg()
  File "c:\users\martin\dev\_pyvenvs\latest36\lib\site-packages\ibapi\connection.py", line 99, in recvMsg
    buf = self._recvAllMsg()
  File "c:\users\martin\dev\_pyvenvs\latest36\lib\site-packages\ibapi\connection.py", line 119, in _recvAllMsg
    buf = self.socket.recv(4096)
OSError: [WinError 10038] Ein Vorgang bezog sich auf ein Objekt, das kein Socket ist
